In [2]:
import re
import os
import requests
import subprocess as sp
import pandas as pd
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
#find entries which have healpix or locmap
data = pd.DataFrame(pd.read_csv("/home/ravioli/astro/git/fermi_grbs/data_status.csv"))
print(len(np.argwhere(np.logical_or(data['healpix'].values=='0',data['locmap'].values=='0'))))

1751


In [2]:
# #downloading all healpix and locprob files
# df = pd.DataFrame(pd.read_csv("/home/ravioli/astro/git/fermi_grbs/u_triggers.csv"))

# with open("data_status.csv", "w") as f:
#     os.chdir("/home/ravioli/astro/git/fermi_grbs/data/")
#     for url, trigger in zip(df['url'].values, df['trigger_no'].values):
#         if url != "None":
#             print(url)
#             grb = url.split("/")[-3]
#             healpix = f"{url}glg_healpix_all_{grb}.fit"
#             locprob = f"{url}glg_locprob_all_{grb}.fit"
#             r_healpix = int(os.system(f"timeout 20s wget -nc {healpix}"))
#             r_locprob = int(os.system(f"timeout 20s wget -nc {locprob}"))
#             if not (r_healpix in [0,2048,256] or r_locprob in [0,2048,256]):
#                 print("Downloading again")
#                 print(url)
#                 r_healpix = os.system(f"timeout 20 wget -nc {healpix}")
#                 r_locprob = os.system(f"timeout 20 wget -nc {locprob}")
#             f.write(f"{grb},{trigger},{r_healpix},{r_locprob}\n")
#             # print(f"{grb},{trigger},{r_healpix},{r_locprob}")
#         else:
#             print(url)
#             f.write(f"{None},{trigger},{None},{None}\n")

# f.close()

http://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/2017/bn170101116/quicklook/
http://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/2017/bn170101374/quicklook/
http://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/2017/bn170101885/quicklook/
http://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/2017/bn170103797/quicklook/
http://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/2017/bn170103867/quicklook/
http://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/2017/bn170106968/quicklook/
http://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/2017/bn170107450/quicklook/
http://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/2017/bn170107763/quicklook/
http://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/2017/bn170109137/quicklook/
http://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/2017/bn170110967/quicklook/
http://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/2017/bn170111256/quicklook/
http://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/2017/bn1

In [8]:
# #getting data products link from GCN
# basepath = "https://gcn.gsfc.nasa.gov/other/"
# with open("u_triggers.txt", "a") as f:
#     for trigger in u_triggers:
#         try:
#             r = requests.get(f"{basepath}/{trigger}.fermi")
#             url = os.path.dirname(re.search("(?P<url>http?://[^\s]+)", r.text).group("url"))+"/"
#             f.write(f"{url}\t{trigger}\n")
#         except:
#             print(f"Trigger {trigger} not found")
#             f.write(f"{None}\t{trigger}\n")
# f.close()

Trigger 508930304 not found
Trigger 553789309 not found
Trigger 558642187 not found
Trigger 567431057 not found
Trigger 647120007 not found
Trigger 687017830 not found
